In [2]:
import numpy as np  
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img  
from keras.models import Sequential  
from keras.layers import Dropout, Flatten, Dense  
from keras import applications  
from keras.utils.np_utils import to_categorical  
import matplotlib.pyplot as plt  
import math  
import cv2  
import scipy as sp
import pandas as pd
import random
import time
import matplotlib.pyplot as plt


In [17]:
def process_data():
    filename= "C:/Users/Komal/Desktop/classes/mlp/Kobe/data.csv"
    raw = pd.read_csv(filename,  index_col=None)
    # drops = ['shot_id', 'team_id', 'team_name', 'shot_zone_area', 'shot_zone_range', 'shot_zone_basic', \
             # 'matchup', 'game_event_id', 'game_id', 'game_date']
    # drops = ['team_id', 'team_name', 'matchup', 'game_id', 'game_date']
    drops = ['team_id', 'team_name', 'game_event_id', 'game_id', 'game_date', 'shot_id']
    for drop in drops:
        raw = raw.drop(drop, 1)
    # categorical_vars = ['action_type', 'combined_shot_type', 'shot_type', 'opponent', 'season']
    # categorical_vars = ['action_type', 'combined_shot_type', 'shot_type', 'opponent', 'season', 'shot_id', 'shot_zone_area', 'shot_zone_range', 'shot_zone_basic', \
    #          'matchup', 'game_event_id', 'game_date']
    categorical_vars = ['action_type', 'combined_shot_type', 'shot_type', 'shot_zone_area', 'shot_zone_range', 'shot_zone_basic', 'opponent', 'season']
    for var in categorical_vars:
        raw = pd.concat([raw, pd.get_dummies(raw[var], prefix=var)], 1)
        raw = raw.drop(var, 1)
    for i, row in enumerate(raw.itertuples(), 1):
        if "@" in row.matchup:
            raw.set_value(row.Index, 'matchup', 0)
        else:
            raw.set_value(row.Index, 'matchup', 1)
    df = raw[pd.notnull(raw['shot_made_flag'])]
    indexOfNull = raw[raw['shot_made_flag'].isnull()].index.tolist()
    return raw, df, indexOfNull

def split_data(raw, df, indexOfNull):
    per_test =  round((15 * len(df))/100)
    n =[0 for i in range(per_test)]
    m = len(indexOfNull)-1
    for z in range(per_test):
        c = indexOfNull[m]+1
        x = raw.iloc[c]
        #print(pd.notnull(x['shot_made_flag']))
        flg = 0
        if(pd.isnull(x['shot_made_flag'])):
            #print(c)
            flg =1
            while flg==1:
                c = c+1
                x = raw.iloc[c]
                if pd.notnull(x['shot_made_flag']):
                    flg =0
        n[z] = c
        m=m-1;
    test_comp = raw.iloc[n]
    test = test_comp.drop('shot_made_flag', 1)
    test_y = test_comp['shot_made_flag']
    df = raw.drop(raw.index[n])
    df= df[pd.notnull(df['shot_made_flag'])]
    # separate df into explanatory and response variables
    train = df.drop('shot_made_flag', 1)
    train_y = df['shot_made_flag']
    return train, train_y, test, test_y

In [4]:
raw, df, indexOfNull = process_data()
x_train, y_train, x_test, y_test = split_data(raw, df, indexOfNull)
#print(x_train.shape[1])

C:\Users\Komal\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
C:\Users\Komal\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


In [5]:
from keras import regularizers
def build_model():
    num_sensors=1
    TIME_PERIODS = 146
    num_classes = 1

    model_m = Sequential()

    model_m.add(Reshape((TIME_PERIODS, num_sensors), input_shape=(x_train.shape[1],)))

    model_m.add(Conv1D(200, 73, activation='tanh', input_shape=(TIME_PERIODS, num_sensors), kernel_regularizer=regularizers.l2(0.01)))
    model_m.add(Conv1D(90, 55, activation='tanh'))
    model_m.add(MaxPooling1D(2,2))
    model_m.add(Dropout(0.25))

    model_m.add(Conv1D(300, 10, activation='tanh'))
    model_m.add(Conv1D(110, 1, activation='tanh'))
    model_m.add(MaxPooling1D(1,1))
    model_m.add(Dropout(0.25))

    model_m.add(Conv1D(110, 1, activation='tanh'))
    model_m.add(Conv1D(80, 1, activation='tanh'))
    model_m.add(MaxPooling1D(1,1))
    model_m.add(Dropout(0.25))

    model_m.add(Conv1D(80, 1, activation='tanh'))
    model_m.add(Conv1D(40, 1, activation='tanh'))
    model_m.add(MaxPooling1D(1,1))
    model_m.add(Dropout(0.25))

    model_m.add(GlobalAveragePooling1D())

    model_m.add(Dropout(0.5))

    model_m.add(Dense(num_classes, activation='sigmoid', kernel_regularizer=regularizers.l2(0.01)))
    model_m.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])
    print(model_m.summary())
    return model_m

In [6]:
def compute_and_save_model_weights(model_m):
    
    callbacks_list = [
    ModelCheckpoint(
        filepath='best_model.{epoch:02d}-{val_loss:.2f}.h5',
        monitor='val_loss', save_best_only=True),EarlyStopping(monitor='acc', patience=1)
    ]
    BATCH_SIZE = 400
    EPOCHS = 50
    history = model_m.fit(x_train, y_train, batch_size=BATCH_SIZE,epochs=EPOCHS,validation_split=0.2, verbose=1)
    model_m.save_weights('my_model_weights.h5')

In [7]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, GlobalAveragePooling1D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras.layers.core import Reshape
from  keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.utils import to_categorical
import os


## builds vgg9 model architechture
model = build_model()

## checks if pre-calculated weights are stored in a file
exists = os.path.isfile('C:/Users/Komal/Desktop/classes/mlp/Kobe/my_model_weights.h5')
if not exists:
    compute_and_save_model_weights(model) ## If not stored then it computes new weights on the model that was built by training on train data

model.load_weights('my_model_weights.h5') ## loads the weights into the model architecture

model.pop() ## removes the fully connected layers
#model.pop() 

model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])

bottleneck_features_train = model.predict(x_train)
bottleneck_features_test = model.predict(x_test)
print(np.shape(bottleneck_features_train))
np.savetxt('bottleneck_features_train.csv', bottleneck_features_train, delimiter=",")
np.savetxt('bottleneck_features_test.csv', bottleneck_features_test, delimiter=",")

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_1 (Reshape)          (None, 146, 1)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 74, 200)           14800     
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 20, 90)            990090    
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 10, 90)            0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 10, 90)            0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 1, 300)            270300    
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 1, 110)            33110     
__________

In [9]:
filename= "C:/Users/Komal/Desktop/classes/mlp/Kobe/bottleneck_features_train.csv"
train = pd.read_csv(filename, header=None)

filename1= "C:/Users/Komal/Desktop/classes/mlp/Kobe/bottleneck_features_test.csv"
test = pd.read_csv(filename1, header=None)

#print(train)

In [10]:
def logloss(act, pred):

    epsilon = 1e-15

    pred = sp.maximum(epsilon, pred)

    pred = sp.minimum(1-epsilon, pred)

    ll = sum(act*sp.log(pred) + sp.subtract(1,act)*sp.log(sp.subtract(1,pred)))

    ll = ll * -1.0/len(act)

    return ll

In [11]:
def find_RandomForest_parameters(train, plot=False):

    # find the best n_estimators for RandomForestClassifier

    print('Finding best n_estimators for RandomForestClassifier...')

    min_score = 100000

    best_n = 0

    scores_n = []

    range_n = np.logspace(0,2,num=3).astype(int)



    for n in range_n:

        print("the number of trees : {0}".format(n))

        t1 = time.time()



        rfc_score = 0.

        rfc = RandomForestClassifier(n_estimators=n)

        for train_k, test_k in KFold(len(train), n_folds=10, shuffle=True):

            rfc.fit(train.iloc[train_k], y_train.iloc[train_k])

            #rfc_score += rfc.score(train.iloc[test_k], train_y.iloc[test_k])/10

            pred = rfc.predict(train.iloc[test_k])

            rfc_score += logloss(y_train.iloc[test_k], pred) / 10

        scores_n.append(rfc_score)

        if rfc_score < min_score:

            min_score = rfc_score

            best_n = n



        t2 = time.time()

        print('Done processing {0} trees ({1:.3f}sec)'.format(n, t2-t1))

    print(best_n, min_score)



    # find best max_depth for RandomForestClassifier

    print('Finding best max_depth for RandomForestClassifier...')

    min_score = 100000

    best_m = 0

    scores_m = []

    range_m = np.logspace(0,2,num=3).astype(int)

    for m in range_m:

        print("the max depth : {0}".format(m))

        t1 = time.time()



        rfc_score = 0.

        rfc = RandomForestClassifier(max_depth=m, n_estimators=best_n)

        for train_k, test_k in KFold(len(train), n_folds=10, shuffle=True):

            rfc.fit(train.iloc[train_k], y_train.iloc[train_k])

            #rfc_score += rfc.score(train.iloc[test_k], train_y.iloc[test_k])/10

            pred = rfc.predict(train.iloc[test_k])

            rfc_score += logloss(y_train.iloc[test_k], pred) / 10

        scores_m.append(rfc_score)

        if rfc_score < min_score:

            min_score = rfc_score

            best_m = m



        t2 = time.time()

        print('Done processing {0} trees ({1:.3f}sec)'.format(m, t2-t1))

    print(best_m, min_score)



    if (plot):

        plot_RandomForest_parameters(range_n, scores_n, range_m, scores_m)



    return best_n, best_m

In [12]:
def plot_RandomForest_parameters(range_n, scores_n, range_m, scores_m):

    plt.figure(figsize=(10,5))

    plt.subplot(121)

    plt.plot(range_n, scores_n)

    plt.ylabel('score')

    plt.xlabel('number of trees')



    plt.subplot(122)

    plt.plot(range_m, scores_m)

    plt.ylabel('score')

    plt.xlabel('max depth')



    plt.show()


In [13]:
def run_RandomForest(train, train_y, test, best_n, best_m):

    model = RandomForestClassifier(n_estimators=best_n, max_depth=best_m)

    model.fit(train, train_y)

    # pred_prob = model.predict_proba(train)

    # pred_tes_probt = model.predict_proba(test)





    pred_train = model.predict(train)

    # pred = model.predict(submission)

    pred_test = model.predict(test)



    return pred_train, pred_test



In [14]:
def predictions_result(pred_train, train_y, pred_test, test_y):

    # Calculate the absolute errors

    errors = abs(pred_train - train_y)

    # Print out the mean absolute error (mae)

    print('Mean Absolute Train Error:', round(np.mean(errors), 2), 'degrees.')



    # Calculate mean absolute percentage error (MAPE)

    mape = 100 * (errors)

    # Calculate and display accuracy

    accuracy = 100 - np.mean(mape)

    print('Train Accuracy:', round(accuracy, 2), '%.')





    # Calculate the absolute errors

    errors1 = abs(pred_test - test_y)

    # Print out the mean absolute error (mae)

    print('Mean Absolute Test Error:', round(np.mean(errors1), 2), 'degrees.')



    # Calculate mean absolute percentage error (MAPE)

    mape = 100 * (errors1)

    # Calculate and display accuracy

    accuracy1 = 100 - np.mean(mape)

    print('Test Accuracy:', round(accuracy1, 2), '%.')



In [15]:
import scipy as sp
import pandas as pd
import random
import time
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import KFold   #from sklearn.model_selection import KFold

best_n, best_m = find_RandomForest_parameters(train)

pred_train, pred_test = run_RandomForest(train, y_train, test, best_n, best_m)

C:\Users\Komal\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Finding best n_estimators for RandomForestClassifier...
the number of trees : 1
Done processing 1 trees (1.031sec)
the number of trees : 10
Done processing 10 trees (9.083sec)
the number of trees : 100
Done processing 100 trees (82.954sec)
100 10.494211345901112
Finding best max_depth for RandomForestClassifier...
the max depth : 1
Done processing 1 trees (8.325sec)
the max depth : 10
Done processing 10 trees (46.432sec)
the max depth : 100
Done processing 100 trees (88.331sec)
10 9.775495554560752


In [16]:
predictions_result(pred_train, y_train, pred_test, y_test)

Mean Absolute Train Error: 0.23 degrees.
Train Accuracy: 76.54 %.
Mean Absolute Test Error: 0.36 degrees.
Test Accuracy: 64.18 %.
